참조 : https://github.com/mklimasz/TransE-PyTorch/blob/master/model.py

In [1]:
import numpy as np 
import torch
import torch.nn as nn 

In [2]:
class TransE(nn.Module):

    def __init__(self, entity_count, relation_count, device = 'cpu', d_norm = 1, embedding_dimension = 100, gamma=1.0):
        super(TransE, self).__init__()
        self.entity_count = entity_count
        self.relation_count = relation_count
        self.device = device
        self.d_norm = d_norm
        self.embedding_dimension = embedding_dimension
        self.gamma = gamma 

        self.entities_embedding = self._init_enitity_embedding()
        self.relations_embedding= self._init_relation_embedding()

        # nn.MarginRankingLoss는 loss(x1,x2,y) = max(0, - y * (x1 - x2) + margin)을 의미 
        self.criterion = nn.MarginRankingLoss(gamma = gamma, reduction='none')



    def _init_enitity_embedding(self):
        entities_embedding = nn.Embedding(num_embeddings=self.entity_count + 1,
                                    embedding_dim=self.embedding_dimension,
                                    padding_idx=self.entity_count)
        # 균등 분포 만들기 
        uniform_range = 6 / np.sqrt(self.embedding_dimension)
        # 균등 분포에서 가져온 임의 값으로 가중치 행렬을 초기화 
        entities_embedding.weight.data.uniform_(-uniform_range, uniform_range)

        # num_embeddings * embedding_dim 행렬이 아웃풋으로 출력 
        return entities_embedding                                                          


    def _init_relation_embedding(self):
        relations_embedding = nn.Embedding(num_embeddings=self.relation_count + 1,
                                     embedding_dim=self.embedding_dimension,
                                     padding_idx=self.relation_count)
        # 균등 분포 만들기 
        uniform_range = 6 / np.sqrt(self.embedding_dimension)
        # 균등 분포에서 가져온 임의 값으로 가중치 행렬을 초기화 
        relations_embedding.weight.data.uniform_(-uniform_range, uniform_range)
        
        # -1 to avoid nan for OOV vector
        # Relations의 행을 L1 정규화 
        relations_embedding.weight.data[:-1, :].div_(relations_embedding.weight.data[:-1, :].norm(p=1, dim=1, keepdim=True))                
        return relations_embedding


    def forward(self, original_triplets: torch.LongTensor, corrupted_triplets: torch.LongTensor):

        # -1 to avoid nan for OOV vector
        # Entities의 행을 L2 정규화 
        self.entities_embedding.weight.data[:-1, :].div_(self.entities_embedding.weight.data[:-1, :].norm(p=2, dim=1, keepdim=True))   
       
        original_distances = self._distance(original_triplets)
        corrupted_distances = self._distance(corrupted_triplets)

        return self.loss(original_distances, corrupted_distances), original_distances, corrupted_distances


    def predict(self, triplets: torch.LongTensor):
        return self._distance(triplets)


    def loss(self, original_distances, corrupted_distances):
        target = torch.tensor([-1], dtype=torch.long, device=self.device)
        return self.criterion(original_distances, corrupted_distances, target)


    def _distance(self, triplets):
        heads = triplets[:, 0]
        relations = triplets[:, 1]
        tails = triplets[:, 2]

    # 임베딩 공간 속 h + l과 t의 거리 구하기   
        return (self.entities_embedding(heads) + self.relations_embedding(relations) - self.entities_embedding(tails)).norm(p=self.d_norm, dim=1)

결국 optimize은 임베딩의 가중치를 최적화하는 과정 
"h + l = t에 근사하는 임베딩 가중치를 얻겠다"